In [1]:
import numpy as np
from ortools.algorithms import pywrapknapsack_solver


In [146]:
# Generate input sequences
def gen_inputs(T, weight_low, weight_high, p_min, p_max):
    weights = []
    inner_weights=np.zeros(T)
    for i in range(T):
        tmp=int(np.random.uniform(low=weight_low, high=weight_high))
        while tmp==0:
            tmp=int(np.random.uniform(low=weight_low, high=weight_high))
        inner_weights[i] = tmp
    weights.append(list(inner_weights))
    
    dens = np.random.uniform(low=p_min, high=p_max, size=T)
    values = list(np.multiply(weights[0],dens).astype(int))
    # print(weights[:10])
    # print(dens[:10])
    # print(values[:10])
    return values, weights

## Online Threshold Solver

In [147]:
def get_phi(y, beta, p_min, capacities):
    if y < beta:
        return p_min
    elif beta <= y and y <= capacities[0]:
        return p_min * np.exp(y / beta - 1)
    else: 
        print("ERROR! y exceeds max capacity!")

def get_total_vals(values, x_ts):
    total_val = 0
    for i in range(len(x_ts)):
        total_val += values[i]*x_ts[i]
    return total_val

def get_total_cap(weights, x_ts):
    total_cap = 0
    for i in range(len(x_ts)):
        total_cap += weights[0][i]*x_ts[i]
    return total_cap

In [148]:
def threshold_solver(values, weights, capacities, p_min, p_max):
    sigma=10**-10 # handle zero weight

    beta=1/(1+np.log(p_max/p_min))
    beta=beta*capacities[0]

    # print('beta', beta)
    T = len(values)
    y_ts = np.zeros(T)
    x_ts = np.zeros(T)
    load = 0
    
    for t in range(T):
        value = values[t]
        weight = weights[0][t]
        if weight == 0:
            weight += sigma
        val_density = value/weight
        
        # get p_t
        p_t = p_min
        if t > 0:
            p_t = get_phi(y_ts[t-1], beta, p_min, capacities)
        # print('t:', t, ' p_t:', p_t, ' val_den:', val_density)
        
        # accept        
        if val_density >= p_t and y_ts[t-1] + weight <= capacities[0]:
            x_ts[t] = 1
        # deny
        else:
            x_ts[t] = 0
            
        if t==0:
            y_ts[t] = weight*x_ts[t]

        else:
            y_ts[t] = y_ts[t-1] + weight*x_ts[t]
        
        # capacity is full, end loop
        if y_ts[t] == capacities[0]:
            break;
    alpha=beta
    return y_ts, x_ts, alpha

In [149]:
 def get_online_res(values, weights, capacities, p_min, p_max):
    # min_den=10**10
    # max_den=0
    # sigma=10**-10
    # for i in range(len(values)):
    #     tmp_den = values[i]/(weights[0][i])
    #     # print(tmp_den)
    #     min_den = min(min_den, tmp_den)
    #     max_den = max(max_den, tmp_den)
    # print(min_den)
    # print(max_den)

#     p_min=10
#     p_max= max_den+10

    y_ts, x_ts, alpha = threshold_solver(values, weights, capacities, p_min, p_max)
    total_val = get_total_vals(values, x_ts)
    total_cap=get_total_cap(weights, x_ts)
    # print('total value:', total_val)
    # print('total weight:', total_cap)
    # print('alpha:', alpha)
    return total_val, total_cap, alpha, x_ts, y_ts
    # print('y_ts:', y_ts)
    # print('x_ts:', x_ts)

In [150]:
# print(1/(1+np.log(p_max/p_min)))
# print(np.log(p_max/p_min))

## Offline Solver from OR-Tools

In [152]:
# branch and bound solver
def get_offline_res(values, weights, capacities):
    solver = pywrapknapsack_solver.KnapsackSolver(
    pywrapknapsack_solver.KnapsackSolver.
    KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')
    
    solver.Init(values, weights, capacities)
    computed_value = solver.Solve()
    packed_items = []
    packed_weights = []
    total_weight = 0
    # print('Total value =', computed_value)
    for i in range(len(values)):
        if solver.BestSolutionContains(i):
            packed_items.append(i)
            packed_weights.append(weights[0][i])
            total_weight += weights[0][i]
    # print('Total weight:', total_weight)
    # print('Packed items:', packed_items)
    # print('Packed_weights:', packed_weights)
    return computed_value, total_weight, packed_items

## Infinitesimal Setting (Q1)

In [39]:
# # infinitesimal setting
weight_highs=np.ones(10)
for i in range(len(weight_highs)):
    # weight_highs[i] = 1e-10 * 10**i
    weight_highs[i] = 10 * 10**i
# weight_highs = np.linspace(1, 100, num=10, endpoint=True)

print(weight_highs)
    
cr_list_highs=[]
for weight_high in weight_highs:
    cr_list=[]
    for itrs in range(100):
        T=100

        # infinitesimal setting
        p_min= 1
        p_max= 100
        weight_low = 1
        # weight_high = 1e-1

        values, weights = gen_inputs(T=T, weight_low=weight_low, weight_high=weight_high, p_min=p_min, p_max=p_max)
        # print(values[:20])
        # print(weights[:20])
        # capacities=[max(1, int(np.sum(weights)/2))]
        capacities=[10**10]

        # print('Capacities:', capacities)

        total_val, total_cap, alpha, x_ts, y_ts = get_online_res(values, weights, capacities, p_min, p_max)
        packed_idx=[i for i, e in enumerate(x_ts) if e != 0]

        computed_value, total_weight, packed_items = get_offline_res(values, weights, capacities)
        # print(total_cap)
        # print(total_weight)
        cr = total_cap/total_weight
        cr_list.append(cr)
    cr_list_highs.append(cr_list)
        # print('Empirical C.R.:', cr)
        # print('Theoretical C.R.:', alpha)
        # print(np.array_equal(packed_idx, packed_items))

[1.e+01 1.e+02 1.e+03 1.e+04 1.e+05 1.e+06 1.e+07 1.e+08 1.e+09 1.e+10]


In [40]:
cr_means=[]
cr_stds=[]
for i in range(len(cr_list_highs)):
    cr_list = cr_list_highs[i]
    # print('Trial', i)
    # print('  ', cr_list)
    # print('  ', np.mean(cr_list))
    # print('  ', np.std(cr_list))
    cr_means.append(np.mean(cr_list))
    cr_stds.append(np.std(cr_list))
    

In [41]:
print(cr_means)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9885968121504445,
 0.9917177510421179,
 0.9751310345567551]

In [42]:
print(cr_stds)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.012170832497202029,
 0.006301298229186578,
 0.02004519065184244]

## Non-infinitesimal Setting (Q2)

In [230]:
# non-infinitesimal setting
weight_lows=np.linspace(10**9, 10**10, num=10, endpoint=True)

# weight_lows=np.ones(10)
# for i in range(len(weight_lows)):
#     weight_lows[i] = 10 * 10**i
print(weight_lows)

# weight_lows=[10**(-3)]

cr_list_highs=[]
for weight_low in weight_lows:
    cr_list=[]
    for itrs in range(100):
        T=100

        # non-infinitesimal setting
        p_min= 1
        p_max= 100
        # weight_low = 1e-2
        weight_high = 10**10

        values, weights = gen_inputs(T=T, weight_low=weight_low, weight_high=weight_high, p_min=p_min, p_max=p_max)
        # capacities=[max(1, int(np.sum(weights)/2))]
        capacities=[10**10]
        # p_min= 1
        # p_max= 300099
        # print('Capacities:', capacities)

        total_val, total_cap, alpha, x_ts, y_ts = get_online_res(values, weights, capacities, p_min, p_max)
        packed_idx=[i for i, e in enumerate(x_ts) if e != 0]
        # print(total_cap)
        # print(x_ts)

        computed_value, total_weight, packed_items = get_offline_res(values, weights, capacities)
        # print(packed_items)
        # print(total_weight)
        cr = total_cap/total_weight
        cr_list.append(cr)
    cr_list_highs.append(cr_list)
        # print('Empirical C.R.:', cr)
        # print('Theoretical C.R.:', alpha)
        # print(np.array_equal(packed_idx, packed_items))

[1.e+09 2.e+09 3.e+09 4.e+09 5.e+09 6.e+09 7.e+09 8.e+09 9.e+09 1.e+10]


In [231]:
cr_means=[]
cr_stds=[]
for i in range(len(cr_list_highs)):
    cr_list = cr_list_highs[i]
    # print('Trial', i)
    # print('  ', cr_list)
    # print('  ', np.mean(cr_list))
    # print('  ', np.std(cr_list))
    cr_means.append(np.mean(cr_list))
    cr_stds.append(np.std(cr_list))
    

In [233]:
print(cr_means)

[0.9683662843215227,
 0.9291530801370044,
 0.9091164642807523,
 0.8652210891854952,
 0.792163345730828,
 0.8409095938799466,
 0.8790134913321457,
 0.9224685818398173,
 0.96656832180425,
 1.0]

In [234]:
print(cr_stds)

[0.03609225944537068,
 0.06793634979486571,
 0.0903863565293656,
 0.13283824147256487,
 0.1582690300706349,
 0.13509511087369852,
 0.0934878640705934,
 0.06272543666871325,
 0.0339341597867519,
 0.0]

In [ ]:
# for i in range(len(x_ts)):
#     if x_ts[i]==1:
#         print(p_ts[i], values[i]/weights[0][i], y_ts[i])
        
# print('--')
# weight_tmp=0
# for j in packed_items:
#     weight_tmp+=weights[0][j]
#     print(p_ts[j], values[j]/weights[0][j], weight_tmp)

In [ ]:
# print(packed_items)
# # for i in packed_items:
# #     print(packed_items[i])
# #     print(values[packed_items[i]]/weights[0][packed_items[i]])

In [ ]:
# cr_means

In [ ]:
# cr_stds